In [14]:
import cv2
import numpy as np
import os
from glob import glob

# === Settings ===
input_folder = 'combined_images'  # <--- change this
output_folder = 'particles_output'
output_csv = 'particles.csv'
output_size = (64, 64)
min_width = 35
min_height = 35
bbox_padding = 1
upscale = 5

# Create output dir
os.makedirs(output_folder, exist_ok=True)

# Sharpening kernel
sharpen_kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])

# Load all image paths
image_paths = glob(os.path.join(input_folder, '*'))

# To store all flattened particle data
all_particle_data = []
particle_index = 0

# === Process each image ===
for img_path in image_paths:
    image = cv2.imread(img_path)
    if image is None:
        print(f"Skipping {img_path} (not a valid image)")
        continue

    img_height, img_width = image.shape[:2]

    # HSV blue mask
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([110, 180, 80])
    upper_blue = np.array([130, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        if w < min_width or h < min_height:
            continue

        # Apply 1-px padding
        x_pad = max(x - bbox_padding, 0)
        y_pad = max(y - bbox_padding, 0)
        x_end = min(x + w + bbox_padding, img_width)
        y_end = min(y + h + bbox_padding, img_height)

        # Crop and enlarge
        particle = image[y_pad:y_end, x_pad:x_end]
        enlarged = cv2.resize(particle,
                              ((x_end - x_pad) * upscale, (y_end - y_pad) * upscale),
                              interpolation=cv2.INTER_CUBIC)

        # Grayscale and sharpen
        gray = cv2.cvtColor(enlarged, cv2.COLOR_BGR2GRAY)
        sharpened = cv2.filter2D(gray, -1, sharpen_kernel)

        # Resize while keeping aspect ratio
        h_i, w_i = sharpened.shape
        scale = min(output_size[0] / h_i, output_size[1] / w_i)
        new_w, new_h = int(w_i * scale), int(h_i * scale)
        resized = cv2.resize(sharpened, (new_w, new_h), interpolation=cv2.INTER_AREA)

        # Pad to 64x64 with white background
        padded = np.full(output_size, 255, dtype=np.uint8)
        x_offset = (output_size[1] - new_w) // 2
        y_offset = (output_size[0] - new_h) // 2
        padded[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

        # Save particle image
        out_path = os.path.join(output_folder, f"particle_{particle_index:04d}.png")
        cv2.imwrite(out_path, padded)

        # Add to dataset
        all_particle_data.append(padded.flatten())
        particle_index += 1

print(f"Extracted {particle_index} particles from {len(image_paths)} images.")

# Save final CSV
all_particle_array = np.array(all_particle_data)
np.savetxt(output_csv, all_particle_array, fmt='%d', delimiter=',')

print(f"All particle data saved to: {output_csv}")


Extracted 18252 particles from 574 images.
All particle data saved to: particles.csv
